# 基于futures的并发


本章的中文标题翻译似乎略有问题：原书本章的标题为Concurrency with futures，即利用futures进行并发，本文翻译为使用future处理并发。

本章的第一段，从作者的描述中可以得知：future和futures是两个概念。future是一种对象，其表示异步执行的操作；futures则指的是concurrent.futures模块。future是concurrent.futures以及asyncio包的基础，因此本章应当翻译为"基于futures的并发"或者"利用futures处理并发"。

本章除了标题外，正文部分有错误。第426页，"职程"应当是"进程"。本人使用的书是2021年10月河北第33次印刷，ISBN：978-7-115-45415-7。

## 基于concurrent.futures的多线程

相较于CPU密集型任务，Python更适合执行I/O密集型任务的并行。对于I/O密集型任务，Python仅需要维护一堆线程，然后使用队列收集这些线程的结果即可。

concurrent.futures提供了线程池，即concurrent.futures.ThreadPoolExecutor，线程池能够维护一定数量的线程，然后执行并发。基本使用方法如下：

```Python
with futures.ThreadPoolExecutor(workers) as executor:
    res = executor.map(func, iterable_obj)
```
上述伪代码描述了使用ThreadPoolExecutor进行并发的基础方法。with语句如15章描述的那样，在代码片段开始前/后会执行上下文管理器的特定方法。在这里，executor.\_\_enter\_\_会创建线程，executor.\_\_exit\_\_则会调用executor.shutdown(wait=True)方法，以便在所有线程执行完毕前阻塞线程。

executor.map则会在多个线程中并发调用func执行具体的功能

## concurrent.futures背后的future

future是一种对象，其封装了**待完成**的操作，并可以作为基本元素放入队列。future的状态可以查询，其结果可以获取。

从上述描述可以发现，单纯一个future对象，其中的操作何时启动，何时完成均不确定。就好像是把做菜需要的工具（函数）以及材料（数据）打包，但是厨师何时做菜以及何时做完均是未知的，作为前台服务员仅能知道厨师有没有完成（查询状态），并且在完成后获得做好的菜（获得结果）或是厨师抱怨给的东西有问题（抛出错误）。因此future需要和其他框架配合使用，例如上述的concurrent.futures。

executor.map执行了多项重要工作：
1. 根据func以及iterable_obj创建一堆future对象，这些对象表示需要完成的任务
2. 排定所有future对象的顺序（执行时间）
3. 交由各线程执行
4. 接收并返回一个迭代器，该迭代器的\_\_next\_\_方法会执行各future的result方法

上述工作可手动实现以代替executor.map，其伪代码如下
```Python
with futures.ThreadPoolExecutor(max_workers) as executor:
    # 初始化记录future的列表
    to_do = list()
    # 创建并排定所有的future
    for _item in sorted(iterable_obj):
        # 根据功能函数func以及元素生成future对象
        _future = executor.submit(func, _item)
        to_do.append(_future)

    # 执行并获取返回值
    result = list()
    for _future in futures.as_completed(to_do):
        res = _future.result()
        results.append(res)
```

上述伪代码创建并执行了各future。其中executor.submit()会对传入的可调用对象进行排定并返回一个future对象；futures.as_completed()则在future运行结束后产出一个future

## 并发、并行以及GIL

并发并不是并行，上述讨论的是并发，而不是并行。并发不一定并行，即上述代码虽然看起来多件任务是同时被处理，但是每一个时刻实际上仅执行了一个任务，通过时间片轮转使得多个任务看起来是同时进行。

由于CPython解释器本身不是线程安全的，因此有全局解释器锁（GIL），仅有获得GIL的线程才能够执行Python字节码。因此Python的多线程对CPU密集型任务尤其不友好 —— 每一时刻有且仅有一个线程在运行。对于CPU密集型任务，无论按照怎样的顺序执行各任务，只要每一个时刻仅能有一个线程在运行那么总运行时间不变。

但是对于I/O密集型任务，当I/O阻塞时，该线程会自动释放GIL，使得其他线程能够执行。相较于一个线程顺序执行，使用多线程显然能够节省下I/O阻塞的时间，这一特点使得Python的多线程在I/O密集型任务中有用武之地

## 基于concurrent.futures的多进程

Python的多进程能够真正实现并行计算，其能够绕过GIL。与多线程类似，concurrent.futures也运行维护有多个进程的进程池 —— ProcessPoolExecutor。ProcessPoolExecutor的用法类似于上述的ThreadPoolExecutor，不同的是，ProcessPoolExecutor会默认使用CPU数量作为最大进程数，而ThreadPoolExecutor需要手动设定线程数。

## map和submit/as_completed组合的对比

.map非常简便易用，只要传入相应的功能函数以及一个可迭代对象就能够一行代码实现多线程/多进程。但是.map的缺陷也很明显，一方面没有方法简便处理多个功能函数（例如一部分对象使用功能函数A，另一部分对象使用功能函数B）；另一方面，参数传递也略死板，当功能函数要求传入多个变量时不方便；此外，.map最致命的问题是：其返回对象是一个迭代器并且其结果的返回顺序和传入的可迭代对象中元素的排序一致，这意味着若第一个元素的运行意外阻塞了，所有的结果均无法获得（虽然执行是并发/并行的，但是结果获取是顺序的）。当然，总是顺序获得结果在一些应用中是潜在漏洞，但是在另一些应用中很可能是有用的（例如应用要求在获得所有结果后再执行后续操作，那么完全可以有意使得可迭代对象中第一个元素的运行阻塞，最后再一并获得所有结果）

不同于map，submit/as_completed组合更灵活，基本上能够控制并发/并行的所有关键步骤，能够处理不同的可调用对象以及参数。此外，该组合可以混用多线程和多进程，as_completed接收的future可以来自于不同的executor实例。

## 多进程/多线程中的异常处理

本章以一个国旗下载程序为例展示了多进程以及多线程中的异常处理方法。总的来说，对于.map，需要在可调用对象中捕获相应的错误并进行处理；对于submit/as_completed组合，需要在as_completed后获取future对象返回值时对异常进行处理

## 总结

1. concurrent.futures可以方便实现多线程或者多进程
2. 本章介绍了两种实现多线程/多进程的方法，.map方法简便易用，但是不够灵活；submit/as_completed组合更灵活，但是实现更复杂
3. 并发并不一定并行，由于Python有GIL，多线程并不能实现真正的并行，而仅能实现并发
4. 多进程/多线程同样需要考虑异常处理，不同的实现有不同的处理方法
5. 对于I/O密集型任务，Python的多线程仍有用武之地 —— 能够节省阻塞的时间
6. 对于CPU密集型任务，Python最好是通过多进程来实现并行，但是多进程的启动需要时间，对于非常简单的任务使用多进程很可能吃力不讨好